# Qwen2.5-VL Error Bar Detection Fine-tuning

This notebook fine-tunes Qwen2.5-VL-7B-Instruct for error bar detection in scientific plots.

## Task:
- **Input**: Scientific plot image + data point coordinates (x, y)
- **Output**: Error bar distances (topBarPixelDistance, bottomBarPixelDistance)

## Approach:
- Use Qwen2.5-VL (Vision-Language Model) with LoRA fine-tuning
- Train on labeled data with known error bar distances
- Evaluate on test data

## 1. Install Required Libraries

In [1]:
# Install libraries for VLM fine-tuning
!pip install -U transformers accelerate bitsandbytes -q
!pip install -U peft trl datasets -q
!pip install -U huggingface_hub -q
!pip install pillow pandas tqdm -q
!pip install qwen-vl-utils -q  # For Qwen VL processing

print("All libraries installed successfully!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 79.3 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 30.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 33.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.1.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 5.0.0 which is incompatible.
gradio 5.49.1 requires pydantic<2.12,>=2.0, but you have pydantic 2.12.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.0/557.0 kB 10.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.9/532.9 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

## 2. Import Libraries

In [2]:
import torch
import os
import json
import gc
import numpy as np
from pathlib import Path
from typing import List, Dict, Optional, Tuple
from PIL import Image
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Transformers
from transformers import (
    Qwen2_5_VLForConditionalGeneration,
    AutoProcessor,
    BitsAndBytesConfig,
    TrainingArguments,
)

# PEFT for LoRA
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    TaskType,
    PeftModel,
)

# Datasets
from datasets import Dataset

# HuggingFace Hub
from huggingface_hub import login, HfApi, create_repo

# Check GPU
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

PyTorch version: 2.8.0+cu126
CUDA available: True
GPU: Tesla T4
GPU Memory: 15.8 GB


## 3. Configuration

In [3]:
import os

# ============== DATA PATHS (Kaggle) ==============
BASE_PATH = "/kaggle/input/graph-plots"

# Training data
TRAIN_IMAGES = os.path.join(BASE_PATH, "Train", "images")
TRAIN_LABELS = os.path.join(BASE_PATH, "Train", "labels")  # Ground truth with error bars

# Test data
TEST_IMAGES = os.path.join(BASE_PATH, "Test", "images")
TEST_INPUT_LABELS = os.path.join(BASE_PATH, "Test", "test_labels")  # Input: x,y only
TEST_GROUND_TRUTH = os.path.join(BASE_PATH, "Test", "labels")  # Ground truth with error bars

# ============== MODEL CONFIGURATION ==============
BASE_MODEL = "Qwen/Qwen2.5-VL-7B-Instruct"
NEW_MODEL_NAME = "Chartqwen"

# HuggingFace Hub
HF_USERNAME = "Sayeem26s"
HF_REPO_NAME = "Chartqwen"
HF_REPO_ID = f"{HF_USERNAME}/{HF_REPO_NAME}"

# ============== TRAINING CONFIGURATION ==============
OUTPUT_DIR = "/kaggle/working/qwen_vl_lora_output"
CHECKPOINT_DIR = "/kaggle/working/qwen_vl_checkpoints"

MAX_SEQ_LENGTH = 2048
IMAGE_MAX_SIZE = 768

BATCH_SIZE = 1  # VLM requires small batch size
GRADIENT_ACCUMULATION_STEPS = 16  # Effective batch = 16

LEARNING_RATE = 2e-4  # Slightly higher for faster convergence
NUM_EPOCHS = 1  # 1 epoch for 600 samples
WARMUP_RATIO = 0.03

SAVE_STEPS = 30  # Save every 30 steps
LOGGING_STEPS = 5  # Log frequently

MAX_TRAIN_SAMPLES = 600  # Fast training: ~1-1.5 hours
MAX_GRAD_NORM = 1.0

# ============== LoRA CONFIGURATION ==============
LORA_R = 32
LORA_ALPHA = 64
LORA_DROPOUT = 0.05

# ============== CREATE DIRECTORIES ==============
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# ============== PRINT CONFIGURATION ==============
print("Configuration:")
print(f"  Base Model          : {BASE_MODEL}")
print(f"  New Model           : {NEW_MODEL_NAME}")
print(f"  Train Images        : {TRAIN_IMAGES}")
print(f"  Train Labels        : {TRAIN_LABELS}")
print(f"  Test Images         : {TEST_IMAGES}")
print(f"  HF Repo             : {HF_REPO_ID}")

print("\nTraining Settings:")
print(f"  - Samples           : {MAX_TRAIN_SAMPLES}")
print(f"  - Epochs            : {NUM_EPOCHS}")
print(f"  - Effective Batch   : {BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS}")
print(f"  - Learning Rate     : {LEARNING_RATE}")
print(f"  - Est. Time         : ~1 hour on T4 GPU")

print("\nLoRA Settings:")
print(f"  - Rank: {LORA_R}, Alpha: {LORA_ALPHA}, Dropout: {LORA_DROPOUT}")

Configuration:
  Base Model          : Qwen/Qwen2.5-VL-7B-Instruct
  New Model           : Chartqwen
  Train Images        : /kaggle/input/graph-plots/Train/images
  Train Labels        : /kaggle/input/graph-plots/Train/labels
  Test Images         : /kaggle/input/graph-plots/Test/images
  HF Repo             : Sayeem26s/Chartqwen

Training Settings:
  - Samples           : 600
  - Epochs            : 1
  - Effective Batch   : 16
  - Learning Rate     : 0.0002
  - Est. Time         : ~1 hour on T4 GPU

LoRA Settings:
  - Rank: 32, Alpha: 64, Dropout: 0.05


## 4. Login to HuggingFace Hub

In [4]:
# Login to HuggingFace Hub
try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    HF_TOKEN = user_secrets.get_secret("HF_TOKEN")
    print("HF Token loaded from Kaggle Secrets")
except:
    HF_TOKEN = "YOUR_HF_TOKEN_HERE"  # Replace with your token
    print("Using hardcoded HF token (replace with your actual token)")

login(token=HF_TOKEN)
print(f"Logged in to HuggingFace Hub as: {HF_USERNAME}")

HF Token loaded from Kaggle Secrets
Logged in to HuggingFace Hub as: Sayeem26s


## 5. Load and Prepare Training Data

In [5]:
def load_label_file(json_path: str) -> List[Dict]:
    """Load label JSON file."""
    with open(json_path, 'r') as f:
        return json.load(f)

def get_image_filename(label_filename: str, images_dir: str) -> Optional[str]:
    """
    Find corresponding image file for a label.
    Label: xxxx.json -> Image: xxxx.png or xxxx.jpg
    """
    base_name = label_filename.replace('.json', '')
    
    for ext in ['.png', '.jpg', '.jpeg', '.PNG', '.JPG', '.JPEG']:
        img_path = os.path.join(images_dir, base_name + ext)
        if os.path.exists(img_path):
            return img_path
    return None

def create_training_sample(image_path: str, label_data: List[Dict]) -> Dict:
    """
    Create a training sample from image and labels.
    
    Input format (label_data):
    [
        {
            "label": {"lineName": "..."},
            "points": [
                {"x": 100, "y": 200, "topBarPixelDistance": 10, "bottomBarPixelDistance": 15, ...},
                ...
            ]
        }
    ]
    
    Output format:
    {
        "image_path": "path/to/image.png",
        "input_points": [{"x": 100, "y": 200}, ...],
        "output_points": [{"x": 100, "y": 200, "topBarPixelDistance": 10, "bottomBarPixelDistance": 15}, ...]
    }
    """
    all_input_points = []
    all_output_points = []
    
    for line_data in label_data:
        points = line_data.get('points', [])
        for pt in points:
            # Skip axis labels
            if pt.get('label', '') in ['xmin', 'xmax', 'ymin', 'ymax']:
                continue
            
            x = pt.get('x', 0)
            y = pt.get('y', 0)
            top_dist = pt.get('topBarPixelDistance', 0)
            bottom_dist = pt.get('bottomBarPixelDistance', 0)
            
            all_input_points.append({"x": round(x, 1), "y": round(y, 1)})
            all_output_points.append({
                "x": round(x, 1),
                "y": round(y, 1),
                "topBarPixelDistance": round(top_dist, 1),
                "bottomBarPixelDistance": round(bottom_dist, 1)
            })
    
    return {
        "image_path": image_path,
        "input_points": all_input_points,
        "output_points": all_output_points
    }

def load_all_training_data(labels_dir: str, images_dir: str, max_samples: Optional[int] = None) -> List[Dict]:
    """
    Load all training samples from labels directory.
    """
    samples = []
    label_files = sorted([f for f in os.listdir(labels_dir) if f.endswith('.json')])
    
    if max_samples:
        label_files = label_files[:max_samples]
    
    print(f"Loading {len(label_files)} training samples...")
    
    for label_file in tqdm(label_files):
        try:
            # Find image
            image_path = get_image_filename(label_file, images_dir)
            if not image_path:
                continue
            
            # Load labels
            label_path = os.path.join(labels_dir, label_file)
            label_data = load_label_file(label_path)
            
            # Create training sample
            sample = create_training_sample(image_path, label_data)
            
            # Only include samples with points
            if sample['input_points']:
                samples.append(sample)
                
        except Exception as e:
            print(f"Error loading {label_file}: {e}")
            continue
    
    print(f"Loaded {len(samples)} valid training samples")
    return samples

# Load training data
print("Loading training data...")
training_samples = load_all_training_data(TRAIN_LABELS, TRAIN_IMAGES, MAX_TRAIN_SAMPLES)

# Show sample
if training_samples:
    print(f"\nSample training data:")
    print(f"  Image: {training_samples[0]['image_path']}")
    print(f"  Num points: {len(training_samples[0]['input_points'])}")
    if training_samples[0]['input_points']:
        print(f"  First input: {training_samples[0]['input_points'][0]}")
        print(f"  First output: {training_samples[0]['output_points'][0]}")

Loading training data...
Loading 600 training samples...


100%|██████████| 600/600 [00:18<00:00, 31.71it/s]

Loaded 600 valid training samples

Sample training data:
  Image: /kaggle/input/graph-plots/Train/images/001f3fd0-23eb-4371-a52a-41b9b71c36bf.png
  Num points: 23
  First input: {'x': 177.0, 'y': 94.2}
  First output: {'x': 177.0, 'y': 94.2, 'topBarPixelDistance': 100.7, 'bottomBarPixelDistance': 52.7}


## 6. Load Model and Processor

In [6]:
# 4-bit Quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

print("Loading processor...")
processor = AutoProcessor.from_pretrained(
    BASE_MODEL,
    trust_remote_code=True,
)

# Set pad token
if processor.tokenizer.pad_token is None:
    processor.tokenizer.pad_token = processor.tokenizer.eos_token

print(f"Processor loaded!")

print("\nLoading model with 4-bit quantization...")
print("This may take 3-5 minutes...")

model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
)

print(f"Model loaded!")

# Prepare model for k-bit training
model = prepare_model_for_kbit_training(model)

# Disable cache for training (required for gradient checkpointing)
model.config.use_cache = False

if torch.cuda.is_available():
    print(f"GPU Memory Used: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

Loading processor...


preprocessor_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. 


config.json: 0.00B [00:00, ?B/s]

...

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/729 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/216 [00:00<?, ?B/s]

Model loaded!
GPU Memory Used: 2.67 GB


## 7. Configure LoRA

In [7]:
# Target modules for Qwen2.5-VL
# These are the key attention layers
target_modules = [
    "q_proj",
    "k_proj",
    "v_proj",
    "o_proj",
    "gate_proj",
    "up_proj",
    "down_proj",
]

# LoRA Configuration
lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=target_modules,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

# Apply LoRA
model = get_peft_model(model, lora_config)

# Print trainable parameters
model.print_trainable_parameters()

print(f"\nLoRA Configuration:")
print(f"  Rank (r): {LORA_R}")
print(f"  Alpha: {LORA_ALPHA}")
print(f"  Target modules: {target_modules}")

trainable params: 95,178,752 || all params: 8,387,345,408 || trainable%: 1.1348

LoRA Configuration:
  Rank (r): 32
  Alpha: 64
  Target modules: ['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj']


## 8. Create Training Prompts

In [8]:
SYSTEM_PROMPT = """You are a precise error bar detection system for scientific plots.
Given an image of a scientific plot and data point coordinates, detect the error bars.
For each point, output the pixel distance from the data point to the top and bottom of the error bar.
If no error bar exists for a point, output 0 for both distances."""

def create_input_prompt(input_points: List[Dict]) -> str:
    """
    Create the input prompt with data point coordinates.
    """
    points_str = json.dumps(input_points, indent=2)
    
    prompt = f"""Analyze this scientific plot image and detect error bars for the following data points:

{points_str}

For each point, measure:
- topBarPixelDistance: pixel distance from data point to top of error bar (0 if none)
- bottomBarPixelDistance: pixel distance from data point to bottom of error bar (0 if none)

Output as JSON array:
[
  {{"x": <x>, "y": <y>, "topBarPixelDistance": <top>, "bottomBarPixelDistance": <bottom>}}
]"""
    
    return prompt

def create_output_response(output_points: List[Dict]) -> str:
    """
    Create the expected output response.
    """
    return json.dumps(output_points, indent=2)

print("Prompt functions defined!")

# Test prompts
if training_samples:
    sample = training_samples[0]
    print("\nSample input prompt:")
    print("-" * 60)
    print(create_input_prompt(sample['input_points'][:2]))
    print("-" * 60)
    print("\nSample output response:")
    print(create_output_response(sample['output_points'][:2]))

Prompt functions defined!

Sample input prompt:
------------------------------------------------------------
Analyze this scientific plot image and detect error bars for the following data points:

[
  {
    "x": 177.0,
    "y": 94.2
  },
  {
    "x": 207.2,
    "y": 105.5
  }
]

For each point, measure:
- topBarPixelDistance: pixel distance from data point to top of error bar (0 if none)
- bottomBarPixelDistance: pixel distance from data point to bottom of error bar (0 if none)

Output as JSON array:
[
  {"x": <x>, "y": <y>, "topBarPixelDistance": <top>, "bottomBarPixelDistance": <bottom>}
]
------------------------------------------------------------

Sample output response:
[
  {
    "x": 177.0,
    "y": 94.2,
    "topBarPixelDistance": 100.7,
    "bottomBarPixelDistance": 52.7
  },
  {
    "x": 207.2,
    "y": 105.5,
    "topBarPixelDistance": 98.2,
    "bottomBarPixelDistance": 39.9
  }
]


## 9. Prepare Dataset for Training

In [9]:
def prepare_training_example(sample: Dict, processor) -> Dict:
    """
    Prepare a single training example with image and text.
    """
    # Load and resize image
    image = Image.open(sample['image_path']).convert('RGB')
    if max(image.size) > IMAGE_MAX_SIZE:
        ratio = IMAGE_MAX_SIZE / max(image.size)
        new_size = (int(image.size[0] * ratio), int(image.size[1] * ratio))
        image = image.resize(new_size, Image.BILINEAR)
    
    # Create prompts
    input_prompt = create_input_prompt(sample['input_points'])
    output_response = create_output_response(sample['output_points'])
    
    # Create conversation format
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image},
                {"type": "text", "text": f"{SYSTEM_PROMPT}\n\n{input_prompt}"}
            ]
        },
        {
            "role": "assistant",
            "content": output_response
        }
    ]
    
    # Apply chat template
    text = processor.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )
    
    return {
        "text": text,
        "image": image,
        "image_path": sample['image_path']
    }

# Prepare training data
print("Preparing training examples...")
prepared_data = []

for i, sample in enumerate(tqdm(training_samples)):
    try:
        prepared = prepare_training_example(sample, processor)
        prepared_data.append(prepared)
    except Exception as e:
        print(f"Error preparing sample {i}: {e}")
        continue

print(f"\nPrepared {len(prepared_data)} training examples")

# Show sample
if prepared_data:
    print(f"\nSample text length: {len(prepared_data[0]['text'])} chars")
    print(f"Sample text preview:")
    print(prepared_data[0]['text'][:500])

Preparing training examples...


100%|██████████| 600/600 [00:10<00:00, 59.88it/s]


Prepared 600 training examples

Sample text length: 4298 chars
Sample text preview:
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
<|vision_start|><|image_pad|><|vision_end|>You are a precise error bar detection system for scientific plots.
Given an image of a scientific plot and data point coordinates, detect the error bars.
For each point, output the pixel distance from the data point to the top and bottom of the error bar.
If no error bar exists for a point, output 0 for both distances.

Analyze this scientific plot image and detect error bars for


## 10. Custom Training Loop (for VLM)

In [10]:
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
import random
from tqdm.auto import tqdm as tqdm_auto

def collate_fn(batch, processor):
    """
    Custom collate function for VLM training.
    """
    texts = [item['text'] for item in batch]
    images = [item['image'] for item in batch]

    inputs = processor(
        text=texts,
        images=images,
        padding=True,
        truncation=True,
        max_length=MAX_SEQ_LENGTH,
        return_tensors="pt"
    )

    inputs['labels'] = inputs['input_ids'].clone()
    return inputs


def train_vlm_lora(
    model,
    processor,
    train_data,
    num_epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    save_steps=SAVE_STEPS,
):
    """
    Custom training loop for VLM with LoRA.
    """

    random.shuffle(train_data)

    optimizer = AdamW(model.parameters(), lr=learning_rate)

    total_steps = (len(train_data) // batch_size) * num_epochs // gradient_accumulation_steps
    warmup_steps = int(total_steps * WARMUP_RATIO)

    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=warmup_steps,
        num_training_steps=total_steps
    )

    print("\nTraining Configuration:")
    print(f"  Samples: {len(train_data)} | Epochs: {num_epochs} | Steps: {total_steps}")
    print(f"  Batch: {batch_size} x {gradient_accumulation_steps} = {batch_size * gradient_accumulation_steps}")
    print(f"  LR: {learning_rate} | Warmup: {warmup_steps} steps")

    model.train()
    global_step = 0

    epoch_progress = tqdm_auto(range(num_epochs), desc="Training Epochs", position=0)

    for epoch in epoch_progress:
        print("\n" + "=" * 60)
        print(f"Epoch {epoch + 1}/{num_epochs}")
        print("=" * 60)

        epoch_loss = 0.0
        num_batches = 0

        batch_progress = tqdm_auto(
            range(0, len(train_data), batch_size),
            desc=f"Epoch {epoch + 1} Batches",
            position=1,
            leave=False
        )

        for i in batch_progress:
            batch = train_data[i:i + batch_size]

            try:
                inputs = collate_fn(batch, processor)
                inputs = {k: v.to(model.device) for k, v in inputs.items()}

                outputs = model(**inputs)
                loss = outputs.loss / gradient_accumulation_steps
                loss.backward()

                epoch_loss += loss.item()
                num_batches += 1

                if num_batches % gradient_accumulation_steps == 0:
                    optimizer.step()
                    scheduler.step()
                    optimizer.zero_grad()
                    global_step += 1

                    current_loss = epoch_loss / num_batches
                    batch_progress.set_postfix({
                        'loss': f'{current_loss:.4f}',
                        'step': f'{global_step}/{total_steps}',
                        'lr': f'{scheduler.get_last_lr()[0]:.2e}'
                    })

                    if global_step % LOGGING_STEPS == 0:
                        print(
                            f"Step {global_step}/{total_steps} | "
                            f"Loss: {current_loss:.4f} | "
                            f"LR: {scheduler.get_last_lr()[0]:.2e}"
                        )

                    if global_step % save_steps == 0:
                        checkpoint_path = os.path.join(
                            CHECKPOINT_DIR, f"checkpoint-{global_step}"
                        )
                        model.save_pretrained(checkpoint_path)
                        print(f"Saved checkpoint: {checkpoint_path}")

                del inputs, outputs, loss
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()

            except torch.cuda.OutOfMemoryError:
                print("OOM encountered, skipping batch")
                optimizer.zero_grad()
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
                continue

        avg_epoch_loss = epoch_loss / num_batches if num_batches > 0 else 0.0
        epoch_progress.set_postfix({'epoch_loss': f'{avg_epoch_loss:.4f}'})
        print(f"Epoch {epoch + 1} complete | Avg Loss: {avg_epoch_loss:.4f}")

    print("\n" + "=" * 60)
    print("TRAINING COMPLETE")
    print("=" * 60)

    return model

## 11. Train the Model

In [11]:
# Train
model = train_vlm_lora(
    model=model,
    processor=processor,
    train_data=prepared_data,
    num_epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    save_steps=SAVE_STEPS,
)


Training Configuration:
  Samples: 600 | Epochs: 1 | Steps: 37
  Batch: 1 x 16 = 16
  LR: 0.0002 | Warmup: 1 steps


Training Epochs:   0%|          | 0/1 [00:00<?, ?it/s]


Epoch 1/1


Epoch 1 Batches:   0%|          | 0/600 [00:00<?, ?it/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step 5/37 | Loss: 0.3896 | LR: 1.78e-04
Step 10/37 | Loss: 0.2972 | LR: 1.50e-04
Step 15/37 | Loss: 0.2567 | LR: 1.22e-04
Step 20/37 | Loss: 0.2336 | LR: 9.44e-05
Step 25/37 | Loss: 0.2189 | LR: 6.67e-05
Step 30/37 | Loss: 0.2084 | LR: 3.89e-05
Saved checkpoint: /kaggle/working/qwen_vl_checkpoints/checkpoint-30
Step 35/37 | Loss: 0.2009 | LR: 1.11e-05
Epoch 1 complete | Avg Loss: 0.1983

TRAINING COMPLETE


## 12. Save Final Model

In [12]:
# Save final LoRA adapter
final_checkpoint_path = os.path.join(CHECKPOINT_DIR, "final_lora_adapter")

print(f"Saving LoRA adapter to: {final_checkpoint_path}")
model.save_pretrained(final_checkpoint_path)
processor.save_pretrained(final_checkpoint_path)

print(f"\nCheckpoint saved!")
print(f"Contents:")
for f in os.listdir(final_checkpoint_path):
    size = os.path.getsize(os.path.join(final_checkpoint_path, f)) / 1024
    print(f"  - {f} ({size:.1f} KB)")

Saving LoRA adapter to: /kaggle/working/qwen_vl_checkpoints/final_lora_adapter

Checkpoint saved!
Contents:
  - adapter_model.safetensors (371875.5 KB)
  - tokenizer.json (11154.5 KB)
  - adapter_config.json (1.0 KB)
  - chat_template.jinja (1.0 KB)
  - tokenizer_config.json (0.7 KB)
  - processor_config.json (1.4 KB)
  - README.md (5.1 KB)


## 13. Upload to HuggingFace Hub

In [13]:
# Create repository
api = HfApi()

try:
    create_repo(
        repo_id=HF_REPO_ID,
        repo_type="model",
        private=False,
        exist_ok=True,
    )
    print(f"Repository created/verified: {HF_REPO_ID}")
except Exception as e:
    print(f"Repo note: {e}")

# Push to Hub
print(f"\nUploading to HuggingFace Hub: {HF_REPO_ID}")

model.push_to_hub(
    repo_id=HF_REPO_ID,
    commit_message="Upload fine-tuned Qwen2.5-VL error bar detector",
)

processor.push_to_hub(
    repo_id=HF_REPO_ID,
    commit_message="Upload processor",
)

print(f"\n{'='*60}")
print(f"MODEL UPLOADED: https://huggingface.co/{HF_REPO_ID}")
print(f"{'='*60}")

Repository created/verified: Sayeem26s/Chartqwen

Uploading to HuggingFace Hub: Sayeem26s/Chartqwen


README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            


MODEL UPLOADED: https://huggingface.co/Sayeem26s/Chartqwen


## Summary

This notebook demonstrated:

### Fine-tuning:
- **Model**: Qwen2.5-VL-7B-Instruct (Vision-Language Model)
- **Method**: LoRA with 4-bit quantization
- **Task**: Error bar detection in scientific plots
- **Input**: Image + data point coordinates (x, y)
- **Output**: Error bar distances (topBarPixelDistance, bottomBarPixelDistance)

### Data Format:
- **Training Labels**: JSON with `points` array containing `x`, `y`, `topBarPixelDistance`, `bottomBarPixelDistance`
- **Test Input**: JSON with `image_file` and `data_points` (x, y only)
- **Test Ground Truth**: Same format as training labels

### Key Features:
- Custom training loop for VLM with image inputs
- Memory-efficient training with gradient accumulation
- Checkpointing and HuggingFace Hub upload
- Comprehensive evaluation metrics

### Model Path:
- HuggingFace: `Sayeem26s/qwen-vl-errorbar-detector`